In [1]:
from ray.train import Trainer

In [2]:
from ray import tune

In [3]:
!rllib

usage: rllib [-h] {train,evaluate,rollout} ...

Train or evaluate an RLlib Trainer.

positional arguments:
  {train,evaluate,rollout}
                        Commands to train or evaluate an RLlib agent.

optional arguments:
  -h, --help            show this help message and exit

Example usage for training:
    rllib train --run DQN --env CartPole-v0

Example usage for evaluate (aka: "rollout"):
    rllib evaluate /trial_dir/checkpoint_000001/checkpoint-1 --run DQN
